In [ ]:
# RNA-seq Differential Expression Analysis

## 1. Load Libraries
R
library(DESeq2)
library(tidyverse)
library(EnhancedVolcano)
library(ggplot2)

## 2. Load Input Files
## gene_counts.txt
## runinfo.csv
# delete first row if the .txt file if you get an error
counts_full <- read.delim("statisticalandfunctionalanalysis/fromweek1/gene_counts.txt")

# Keep Geneid and the last columns (15 samples in this example - columns 7 to 21)
counts <- counts_full[, c(1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21)]

# Set Geneid as rownames and remove the Geneid column
rownames(counts) <- counts$Geneid
counts$Geneid <- NULL

# Load the runinfo.csv file
coldata <- read.csv("statisticalandfunctionalanalysis/fromweek1/runinfo.csv", row.names = 1)

# to check if the sample names match
colnames(counts)
rownames(coldata)

# Clean up the column names
colnames(counts) <- gsub(".sorted.bam", "", colnames(counts))
colnames(counts) <- gsub("alignments.", "", colnames(counts))

# Check if names match now
all(colnames(counts) %in% rownames(coldata))

#If this returns TRUE, you're ready to create a DESeq2 object.